# Assignment 5 - Document Retrieval

### <span style="color:red">THIS IS HOMEWORK.</span>
### Due: May 15th, 11:59 pm on Canvas.
> **Copyright ©2019 University of Washington**  

> **Emily Fox**, author of the TuriCreate based version of this assignment.  
> **Sewoong Oh**, instructor of CSE/STAT 416 for Spring 2019.  
> **Joshua Fan**, generously offered a LOT help in debugging and testing.     
> **Anna Neufeld**, generously offered a lot of help in code review.   
> **Hongjun Wu**, revisor of the Scikit-Learn based version of this assignment.  
> **All rights reserved.**

### Clarification
* <span style="color:red">Read & post to this [Assignment 5 Clarifications](https://canvas.uw.edu/courses/1271722/discussion_topics/4817620) if you are not sure of something.</span>
* Many of you have confusion on why some of your homeworks do not get full credit when you think you submitted the correct answer.
    * Well, the answer is simple. If there's Free Response questions, we need to give credit after we manually look at your response.
    * So yeah, don't panic. It's okay.

### Purpose
* When exploring a large set of documents -- such as Wikipedia, news articles, StackOverflow, etc. -- it can be useful to get a list of related material. 
* To find relevant documents you typically:
    * Decide on a notion of similarity
    * Find the documents that are most similar 
 
### Method & Summary
* In the assignment you will:
    * Gain intuition for different notions of similarity and practice finding similar documents. 
    * Explore the tradeoffs with representing documents using raw word counts and TF-IDF
    * Explore the behavior of different distance metrics by looking at the Wikipedia pages most similar to President Obama’s page.

### Permission
* Permission is hereby granted to students registered for University of Washington CSE/STAT 416.
* For use solely during Spring Quarter 2019 for purposes of the course.  
* No other use, copying, distribution, or modification is permitted without prior written consent. 
* Copyrights for third-party components of this work must be honored.  
* Instructors interested in reusing these course materials should contact the author.

> If you have any questions or you are unclear what we are refering to in this homework, please do not hesitate to reach out! Post a discussion and we would love to listen to and answer your questions. Hope you enjoy this assignment 😜 (Well I enjoyed writing it since ya don't get the chance to write emoji into CSE assignment prompt at 4AM very often)

# Nearest Neighbors

### Import necessary packages
* As usual we need to first import the Python packages that we will need.

In [1]:
# Import Statement, by now I guess you kinda know what these are use for? 
import sklearn as sk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Magic Function (Like...magic)
# Interested? Don't know what it means? Read this!
# https://stackoverflow.com/questions/43027980/purpose-of-matplotlib-inline
%matplotlib inline

# Make Pandas print more stuff
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

> Again, stuff wrote in blockquotes is optional. Just something we think is useful to have around.

### Load Wikipedia dataset

* We will be using a dataset of Wikipedia pages. Each element of the dataset consists of a link to the wikipedia article, the name of the person, and the text of the article (in lowercase).  
* This time, since wikipedia is based on articles of some human, it is useful to let index be the names of the person.
* Let's take an overlook at this dataset.

#### Load Data Basics
* First, we will load in all the wikipedia of people data from file using `pd.read_csv()`. 

> If you need the dataset:   
Download it in Canvas under [/files/Assignment Resources/data/](https://canvas.uw.edu/courses/1271722/files/folder/Assignment%20Resources/data).  

> If data failed to load:   
[1] Check if you have the correct file and is in the correct location!   
[2] Make sure you have `people_wiki.csv.csv`, not `people_wiki.csv.gl`!

> Reminders:  
`'./'` means that you are finding a file within the same directory as the notebook.   
`'../'` means that you are looking for a file or folder one hierarchy above.

In [2]:
wiki = pd.read_csv('/data/people_wiki.csv', index_col='name')

* What do the contents look like?

In [3]:
wiki[0:5]

,URI,text
name,,
Digby Morrell,<http://dbpedia.org/resource/Digby_Morrell>,digby morrell born 10 october 1979 is a former...
Alfred J. Lewy,<http://dbpedia.org/resource/Alfred_J._Lewy>,alfred j lewy aka sandy lewy graduated from un...
Harpdog Brown,<http://dbpedia.org/resource/Harpdog_Brown>,harpdog brown is a singer and harmonica player...
Franz Rottensteiner,<http://dbpedia.org/resource/Franz_Rottensteiner>,franz rottensteiner born in waidmannsfeld lowe...
G-Enka,<http://dbpedia.org/resource/G-Enka>,henry krvits born 30 december 1974 in tallinn ...


* What is the overall scale?

In [4]:
print(wiki.shape)

(59071, 2)


## Identify Useful Features
* A lot times, the dataset we get usually contain a huge amount of features(Think of some of the datasets we used before, which has quite a lot of features).
* Remember that adding one more feature means add one more dimension space when computing distances, which sucks since the level of computation difficulity goes up rapidly.
    * This is sometimes called "The Curse of Dimensionality". Ugh! 🤯
* It is up to us to decide which to use.
    * In some cases, we can use feature selection to find out which feature to use.
        * Remember LASSO? LASSO is really good at eliminating features.
        * 🤜 = LASSO, 👿 = Features
        * 🧐🤜👿👻👻👿👻👿👻👻👻👻👻👻
    * In other cases, it is obvious to a human which feature is useful. Yep, we are talking about this dataset.
        * In this case, we have two features in the dataset. Namely, "URL" and "text".
            * Which one is important to us? Obviously url cannot give us much information when it comes to find nearest articles of famous humans, so we can get rid of it.
            * 🧐🤜👿👻

In [5]:
wiki[0:5]

,URI,text
name,,
Digby Morrell,<http://dbpedia.org/resource/Digby_Morrell>,digby morrell born 10 october 1979 is a former...
Alfred J. Lewy,<http://dbpedia.org/resource/Alfred_J._Lewy>,alfred j lewy aka sandy lewy graduated from un...
Harpdog Brown,<http://dbpedia.org/resource/Harpdog_Brown>,harpdog brown is a singer and harmonica player...
Franz Rottensteiner,<http://dbpedia.org/resource/Franz_Rottensteiner>,franz rottensteiner born in waidmannsfeld lowe...
G-Enka,<http://dbpedia.org/resource/G-Enka>,henry krvits born 30 december 1974 in tallinn ...


In [6]:
wiki = wiki['text']

In [7]:
wiki[0:5]

name
Digby Morrell          digby morrell born 10 october 1979 is a former...
Alfred J. Lewy         alfred j lewy aka sandy lewy graduated from un...
Harpdog Brown          harpdog brown is a singer and harmonica player...
Franz Rottensteiner    franz rottensteiner born in waidmannsfeld lowe...
G-Enka                 henry krvits born 30 december 1974 in tallinn ...
Name: text, dtype: object

* We will also save this version of wiki the `original_wiki`, so we may come back later in this assignment.
    * The idea is that we will perform similiar computation processes using two ways of vectorizing.
    * So maintain a copy of the original dataset is useful, you will see what we mean soon.

In [8]:
original_wiki = wiki

* Now that we only have name versus their page, we can finally dig into some sweet nearest neighbor search machine learning. 😏

## Extract word count vectors

We can extract word count vectors using a function called `CountVectorizer()`.
* [sklearn.feature_extraction.text.CountVectorizer()](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

> #### Some more detailed explainations on vectorizing:
* It says in the documentation it "Convert a collection of text documents to a matrix of token counts", but what does this mean?
* Naturally, what we think of doing is to make a dictionary of word counts, 
    * So let's say for "bound le s s boi" you will have a dictionary of {bound:1, le: 1, s:2, boi:1}
    * But imagine we do that for many words in many texts...
        * That is a lot of different vectors in a lot of dictionaries! 
        * 59071 of them! And they take a lot of spaceeeeeeeeeeeeeeeeeee🌌
        * But more importantly, such data structure is very inefficient when it comes to querying.
    * But this does NOT mean it is useless. We will see them later in this homework.
* So to store them efficiently and make them easy to query, we will tokenize these vectors and put them into a matrix.
    * Namely, we don't store individual dictionaries in each row, but make a list of names and align them to a matrix's column. 
    * What does this matrix look like? Think if we have a matrix with each column for a word, and each row for an article, take a second to think how crowded this matrix might look like.
        * Not very croweded, right? 
        * For example, the column for 'obama' will only have a value if that human is obama-related.
        * Say we take Queen Victoria, she doesn't know Obama, she isn't even an American and not to mention she is dead many years before Obama is born, so I guess her 'obama' column very likely be zero. Same for many, many other humans. We call a matrix with many zeroes "sparse". 
        * Still, such a matrix takes up a lot of spaceeeeeee.
            * But it is MUCH easier for our lovely computer to query.
    * Fortunatelly, we have an effecient way to store it in Scikit-Learn's datastructure.
        * How it works? Well, it is complicated, but the main idea is It saves space by not saving zeroes, therefore reduce the amount of space we take in the memory.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(wiki)

>### Let's take a look at what is this matrix.
* As we talked about in the previous blockquote, it is stored in a "sparse matrix" data structure implemented by some smart humans.

In [10]:
count_matrix  

<59071x548429 sparse matrix of type '<class 'numpy.int64'>'
	with 10244028 stored elements in Compressed Sparse Row format>

### While we are here, let's take a look at (some of) the features of the giant matrix as well.
* Let's give it a shot 100000. Never seen this many features in the assignment...
* We try hart to scroll down, but we can't even reach the words start with `c`.
* That's 100000 columns! If we call `print(count_vectorizer.get_feature_names())`and attempt to print all the feature names, python will likely explode...
* Crazy, right?

In [11]:
print(count_vectorizer.get_feature_names()[0:100000])

['00', '000', '0000', '00000', '00000van', '0001', '00014338', '0001sec', '0002', '00026', '0003', '0005', '000577', '0005sec', '0006', '0007', '0007105916', '0007200374', '0007207328', '0007213506', '000721426xhe', '0007a', '000he', '000in', '000m', '000seelenprojekt', '000tnmickushina', '001', '0017', '001cd', '001ehebbm', '002', '0020849605', '0024', '0026183900', '002864574x', '0028659287', '003', '0033', '0034', '0036', '004', '0043', '0046', '004erdemir', '005', '006', '0060222425', '0060628227', '0060628464', '0060669667', '006074393x', '0064', '0066', '007', '0070710481', '0071357440', '0071375627', '0072131772', '0072131896', '0072222611', '0072225351', '0072438886', '007all', '008', '0080', '0080357547', '008after', '009', '00906603', '0091', '0091857112', '0091900255she', '0099416689', '009at', '00a10', '00g', '00s', '00sex', '00sin', '01', '010', '0100', '01000400', '01011001', '01011001i', '0102', '0102928045since', '010397', '01041984', '010602he', '0108', '011', '0110', 

In [12]:
# I know some of you want to try this after I said it...well in default settings Python would explode.
# print(count_vectorizer.get_feature_names())

## Find nearest neighbors

* Now, it's our turn! 🤩
* Let's start by finding the nearest neighbors of the Barack Obama page using the word count vectors to represent the articles and Euclidean distance to measure distance.  For this, again will we use a Scikit-Learn implementation of nearest neighbor search.
* We begin by constructing a model, then fit with the matrix.
* Since we want to see the ten related neighbors of Obama, please set `n_neighbors=10` when you are constructing the model.
    * [sklearn.neighbors.NearestNeighbors](https://scikit-learn.org/stable/modules/neighbors.html)

In [13]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(n_neighbors=10)
model.fit(count_matrix) 

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

### Let's look at the top 10 nearest neighbors by performing the following query.
* Originally, the assignment from last year will just run a block of code and show you the top ten.
* But since many of you told us you'd like to know a bit more what some of the given code means, let's walk through it.

* First, we need to get "Barack Obama".

In [14]:
obama = count_matrix.getrow(wiki.index.get_loc('Barack Obama'))  # First, we find the "Obama Row".
obama

<1x548429 sparse matrix of type '<class 'numpy.int64'>'
	with 270 stored elements in Compressed Sparse Row format>

* Then, let's look at what the `kneighbors()` give us.
    * The first element is distance from other articles, whereas the second element is the index of the word in the matrix.
    * You can read more about kneighbors() here: [kneighbors(X=None, n_neighbors=None, return_distance=True)](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier.kneighbors)

In [15]:
model.kneighbors(obama)

(array([[ 0.        , 33.01514804, 34.3074336 , 35.79106034, 36.06937759,
         36.24913792, 36.27671429, 36.40054945, 36.44173432, 36.83748091]]),
 array([[35817, 24478, 28447, 14754, 35357, 31423, 13229, 36364, 22745,
          7660]]))

* Well then, let's save the distances and indices. 

In [16]:
dist = model.kneighbors(obama)[0]
index = model.kneighbors(obama)[1]

* Now, let's find similar names.
    * Wait! Remember, the wiki data has `names` as index, whereas the matrix has an numerical index.

In [17]:
wiki[0:5]

name
Digby Morrell          digby morrell born 10 october 1979 is a former...
Alfred J. Lewy         alfred j lewy aka sandy lewy graduated from un...
Harpdog Brown          harpdog brown is a singer and harmonica player...
Franz Rottensteiner    franz rottensteiner born in waidmannsfeld lowe...
G-Enka                 henry krvits born 30 december 1974 in tallinn ...
Name: text, dtype: object

* So first, we will need to get the numerical indices back by using `reset_index()`. 
    * It really just adds a numerical index to the dataset.
    * Why? Because we need an index to represent everybody's names.

In [18]:
new_wiki = wiki.reset_index()['name']
new_wiki[0:5]

0          Digby Morrell
1         Alfred J. Lewy
2          Harpdog Brown
3    Franz Rottensteiner
4                 G-Enka
Name: name, dtype: object

* Here, we will use a function called `pandas.Index.flatten()` that breaks an nD array into an 1D array.
    * Note the difference between before flatten and after flatten.
        * It used to be a 2D array `array([[]])`, but we flattened it to be an `array([])`.
        * So we can perform familiar operations with 1D array.
    * [pandas.Index.flatten()](http://pandas.pydata.org/pandas-docs/version/0.14/generated/pandas.Index.flatten.html)

In [19]:
index

array([[35817, 24478, 28447, 14754, 35357, 31423, 13229, 36364, 22745,
         7660]])

In [20]:
flat_index = index.flatten()
flat_index

array([35817, 24478, 28447, 14754, 35357, 31423, 13229, 36364, 22745,
        7660])

* Similarly, we flatten the distances.
    * Now we have our distances and index as two separate 1D array.

In [21]:
distances = dist.flatten()
distances

array([ 0.        , 33.01514804, 34.3074336 , 35.79106034, 36.06937759,
       36.24913792, 36.27671429, 36.40054945, 36.44173432, 36.83748091])

* Now we need to match the names with the distances.
    * Quick reminder: Remember when we talked in the very first quiz section that DataFrames are constructed with Series?
    * Think of series as one individual column.
    * Let's make that column.

In [22]:
series_index = pd.Series(flat_index)

* Now, let's map every index to their names.

In [23]:
names = series_index.map(new_wiki)
names

0                  Barack Obama
1                     Joe Biden
2                George W. Bush
3                   Mitt Romney
4              Lawrence Summers
5                Walter Mondale
6              Francisco Barrio
7                    Don Bonker
8    Wynn Normington Hugh-Jones
9      Refael (Rafi) Benvenisti
dtype: object

* Finally, let's stitch the names with their distances together, and see what the outcome is.

In [24]:
result = pd.DataFrame({'distance':distances, 'name':names})
result

,distance,name
0,0.000000,Barack Obama
1,33.015148,Joe Biden
2,34.307434,George W. Bush
3,35.791060,Mitt Romney
4,36.069378,Lawrence Summers
5,36.249138,Walter Mondale
6,36.276714,Francisco Barrio
7,36.400549,Don Bonker
8,36.441734,Wynn Normington Hugh-Jones
9,36.837481,Refael (Rafi) Benvenisti


* Whoa! That really took quite a while to write, but hopefully this helps.😄

### Now let's take a look at the humans we think is close to Obama!

* All of the 10 people are politicians, but about half of them have rather tenuous connections with Obama, other than the fact that they are politicians.

    * Francisco Barrio is a Mexican politician, and a former governor of Chihuahua.
    * Walter Mondale and Don Bonker are Democrats who made their career in late 1970s.
    * Wynn Normington Hugh-Jones is a former British diplomat and Liberal Party official.
    * Refael (Rafi) Benvenisti is a economist from Israel.

* Nearest neighbors with raw word counts got some things right, showing all politicians in the query result, but missed finer and important details.

* For instance, let's find out why Francisco Barrio was considered a close neighbor of Obama.  To do this, let's look at the most frequently used words in each of Barack Obama and Francisco Barrio's pages:

In [25]:
def top_words(name):
    # Get the most frequent words in the given person's wikipedia page.
    word = count_matrix.getrow(wiki.index.get_loc(name)).toarray().flatten()
    indices = count_vectorizer.get_feature_names()
    words = pd.Series(word, index = indices)
    words = words.sort_values(ascending=False).reset_index()
    words.rename(columns={'index':'word',0:'count'}, inplace=True)
    return words

In [26]:
obama_words = top_words('Barack Obama')
obama_words[0:10]

,word,count
0,the,40
1,in,30
2,and,21
3,of,18
4,to,14
5,his,11
6,obama,9
7,act,8
8,he,7
9,us,6


In [27]:
barrio_words = top_words('Francisco Barrio')
barrio_words[0:10]

,word,count
0,the,36
1,of,24
2,and,18
3,in,17
4,he,10
5,to,9
6,chihuahua,7
7,governor,6
8,as,5
9,his,5


* (chihuahua was mentioned seven times in Barrio's words. What a sweet man!)

## Obama and Barrio comparison

Let's extract the list of most frequent words that appear in both Obama's and Barrio's documents. We've so far sorted all words from Obama and Barrio's articles by their word frequencies. We will now use a dataframe operation known as **join**. The **join** operation is very useful. when it comes to playing around with data: it lets you combine the content of two tables using a shared column (in this case, the word column). See [the documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) for more details.

For instance, running
```
obama_words.join(barrio_words.set_index('word'),rsuffix='_1', on = 'word')
```
will extract the rows from both tables that correspond to the common words.

In [28]:
combined_words = obama_words.join(barrio_words.set_index('word'),rsuffix='_1', on = 'word')
combined_words[0:6]

,word,count,count_1
0,the,40,36
1,in,30,17
2,and,21,18
3,of,18,24
4,to,14,9
5,his,11,5


Since both tables contained the column named `count`, the function we wrote automatically renamed one of them to prevent confusion by adding `rsuffix='_1'`. Let's rename the columns to tell which one is for which. By inspection, we see that the first column (`count`) is Obama and the second (`count.1`) for Barrio.

In [29]:
combined_words.rename(columns={'count':'Obama','count_1':'Barrio'}, inplace=True)
combined_words[0:6]

,word,Obama,Barrio
0,the,40,36
1,in,30,17
2,and,21,18
3,of,18,24
4,to,14,9
5,his,11,5


**Note**. The **join** operation does not enforce any particular ordering on the shared column. So to obtain, say, the five common words that appear most often in Obama's article, sort the combined table by the Obama column. Don't forget `ascending=False` to display largest counts first.

In [30]:
combined_words.sort_values('Obama', ascending = False)[0:5]

,word,Obama,Barrio
0,the,40,36
1,in,30,17
2,and,21,18
3,of,18,24
4,to,14,9


* Remember when we said word count vector dictionaries aren't completely useless a while ago?
    * Turns out it is very useful to construct such a column with dictionaries if we want to compare articles with their common words.
    * Why? 
        * Each document has far less than all the words there is among all documents. It is much easier to compare two small dictionaries if they have common keys.

In [31]:
def word_counter(string):
    counts = dict()
    words = string.split()

    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return counts

wiki = wiki.reset_index()
wiki['word_count'] = wiki['text'].apply(word_counter)

In [32]:
wiki[0:5]

,name,text,word_count
0,Digby Morrell,digby morrell born 10 october 1979 is a former...,"{'digby': 1, 'morrell': 5, 'born': 1, '10': 1,..."
1,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,"{'alfred': 1, 'j': 1, 'lewy': 3, 'aka': 1, 'sa..."
2,Harpdog Brown,harpdog brown is a singer and harmonica player...,"{'harpdog': 2, 'brown': 2, 'is': 7, 'a': 7, 's..."
3,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,"{'franz': 1, 'rottensteiner': 3, 'born': 1, 'i..."
4,G-Enka,henry krvits born 30 december 1974 in tallinn ...,"{'henry': 1, 'krvits': 1, 'born': 1, '30': 1, ..."


Among the words that appear both in Barack Obama and Francisco Barrio pages, take the 5 that appear most frequently in Obama. Identify the articles in the Wikipedia dataset that contain all of these 5 words.

Hint:
* Refer to the previous paragraph for finding the words that appear in both articles. Sort the common words by their frequencies in Obama's article and take the largest five.
* Each word count vector is a Python dictionary. For each word count vector, you'd have to check if the set of the 5 common words is a subset of the keys of the word count vector. Complete the function `has_top_words` to accomplish the task.
  - Convert the top 5 words into set using the syntax
```
set(top_words)
```
    where `top_words` is a Pandas series. 
  - Use [`issubset()` method](https://www.programiz.com/python-programming/methods/set/issubset) to check if all 5 words are among the keys.
* We will apply the `has_top_words` function on every row of the Pandas DataFrame in a bit, and we provided the code for you to apply.
* Compute the sum of the result column to obtain the number of articles containing all the 5 top words.
    * For Q1. 

In [33]:
# ### Write your code here!
combined_words = obama_words.join(barrio_words.set_index('word'),rsuffix='_1', on = 'word')
combined_words.rename(columns={'count':'Obama','count_1':'Barrio'}, inplace=True)
# common_words
# print(combined_words)
common_words = set(combined_words.sort_values('Obama', ascending = False)['word'][0:5])
print(common_words)
def has_top_words(word_count_vector):
    unique_words = word_count_vector.keys()
    if common_words.issubset(unique_words):
        return True
    else:
        return False
    # return True if common_words is a subset of unique_words
    # return False otherwise
#     return ...


{'and', 'in', 'of', 'the', 'to'}


Check your `has_top_words` function on two random articles:
* Write a function called `print_unique()` that takes in a `dict` of `word_count_vector` and prints out the Length of unique words in that word_count_vector.
* Then, use that to check the length of the documents at index `32` and `33`.

In [34]:
print('Output from your function:', has_top_words(wiki.iloc[32]['word_count']))
print('Correct output: True')
print('Also check the length of unique_words. It should be 167.')

Output from your function: True
Correct output: True
Also check the length of unique_words. It should be 167.


In [35]:
# Write your check length of unique words here!
def print_unique(word_count_vector):
    print(len(word_count_vector.keys()))
print_unique(wiki.iloc[32]['word_count'])

167


In [36]:
print('Output from your function:', has_top_words(wiki.iloc[33]['word_count']))
print('Correct output: False')
print('Also check the length of unique_words. It should be 188.')

Output from your function: False
Correct output: False
Also check the length of unique_words. It should be 188.


In [37]:
# Write your check length of unique words here!
print_unique(wiki.iloc[33]['word_count'])

188


* Below is a simple function that applies the function to `word_count` and make a new column called `has_top_words`.

In [38]:
wiki['has_top_words'] = wiki['word_count'].apply(has_top_words)

### **Q1) Among the words that appear both in Barack Obama and Francisco Barrio pages, take the 5 that appear most frequently in Obama. What percentage of the articles in the Wikipedia dataset contain all of these top 5 words? Give answer as a decimal rounded to the nearest 0.0001 - for instance if the percentage found was 19.26% then state answer as 0.1926.** 0.9491
* Fun fact...do you know you can sum up booleans in Python using `sum()` as well?
    * Python will treat True as 1, and False as 0 if it ask it to sum up a bunch of booleans.

In [39]:
# Write your answers here!
wiki['has_top_words'].sum()/len(wiki['has_top_words'])

0.9491290142371045

## Pairwise distances

* Now measure the pairwise distance between the Wikipedia pages of Barack Obama, George W. Bush, and Joe Biden.

* Hint:
    * Before everything else, let's come back to original_wiki, which is the wiki dataframe we first obtained.
        * Why use that wiki? Because you can then easily get the row from the word count matrix we obtained before.
        * We performed similar operations before in this assignment! Remember...how did we find the obama row?
    * To compute the Euclidean distance between two rows, use `sklearn.metrics.pairwise.euclidean_distances`. Refer to [this link](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.euclidean_distances.html) for usage.

In [40]:
# Revert wiki back to the original
wiki = original_wiki
wiki

name
Digby Morrell                               digby morrell born 10 october 1979 is a former...
Alfred J. Lewy                              alfred j lewy aka sandy lewy graduated from un...
Harpdog Brown                               harpdog brown is a singer and harmonica player...
Franz Rottensteiner                         franz rottensteiner born in waidmannsfeld lowe...
G-Enka                                      henry krvits born 30 december 1974 in tallinn ...
Sam Henderson                               sam henderson born october 18 1969 is an ameri...
Aaron LaCrate                               aaron lacrate is an american music producer re...
Trevor Ferguson                             trevor ferguson aka john farrow born 11 novemb...
Grant Nelson                                grant nelson born 27 april 1971 in london also...
Cathy Caruth                                cathy caruth born 1955 is frank h t rhodes pro...
Sophie Crumb                                sophia viol

In [41]:
from sklearn.metrics.pairwise import euclidean_distances
obama = count_matrix.getrow(wiki.index.get_loc('Barack Obama'))
bush = count_matrix.getrow(wiki.index.get_loc('George W. Bush'))
biden = count_matrix.getrow(wiki.index.get_loc('Joe Biden'))
obama_bush = euclidean_distances(obama, bush)
obama_biden = euclidean_distances(obama, biden)
bush_biden = euclidean_distances(bush, biden)
print(obama_bush)
print(obama_biden)
print(bush_biden)

[[34.3074336]]
[[33.01514804]]
[[32.69556545]]


### **Q2) Which of the three pairs has the smallest distance?**

- Obama - Biden 
- Bush - Biden (o)
- Obama - Bush
- All had the same distance

**Note.** Even though common words are swamping out important subtle differences, commonalities in rarer political words still matter on the margin. This is why politicians are being listed in the query result instead of musicians, for example. In the next subsection, we will introduce a different metric that will place greater emphasis on those rarer words.

## TF-IDF to the rescue

* Much of the perceived commonalities between Obama and Barrio were due to occurrences of extremely frequent words, such as "the", "and", and "his". 
* So nearest neighbors is recommending plausible results sometimes for the wrong reasons. 

* To retrieve articles that are more relevant, we should focus more on rare words that don't happen in every article. **TF-IDF** (term frequency–inverse document frequency) is a feature representation that penalizes words that are too common.  Let's use Scikit-Learn's implementation of TF-IDF and repeat the search for the 10 nearest neighbors of Barack Obama:
* I included a longggggggg and detailllllllled description of the given code in the previous section, so I will not explain it again to save space.
    * The Nearest Neighbor and TFIDF vectorization is almost identical. 
    * If you wonder what these given code means, look at the previous section when we vectorized the articles based on word counts.

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

* Now, Try implement your TF-IDF model using the steps we just walked you through.
    * This time, use `TfidfVectorizer()`.
    * Also, pass in `max_df=0.97` when you call `TfidfVectorizer()`.
    * You should obtain a model called `tf_model`, and a matrix called `tfidf_matrix`.
    * The `Extract word count vectors` section we walked through before might be helpful.

### Think for a bit...🤔
* Now take a while and think...why do we pass in `max_df=0.97` into the TfidfVectorizer?
    * If we use the default `max_df=1.00`, we get back ALL of the words and their tf-idf scores.
    * If we use the `max_df=0.97`, this will exclude words that appear in more than 97% of the documents.
* But still, why?
    * What is tf-idf?
        * In a nut shell, it is the *ratio* of the overall frequency to the number of documents it appears in.
    * So what happens if we use all the words?
        * If a word appears very often, it could still have a high tf-idf score even if it also appears in many documents.
            * Namely, 'a', 'the', 'he', etc.
        * Do these common words help when we look at the dataset as a whole?
            * Nope!
            * So we get rid of the words that appear in more than 97% of the articles.
* (Thanks Joshua soooooooooo much on this one!!)

In [47]:
# Write your answer here!
tfidf_vectorizer = TfidfVectorizer(max_df=0.97)
tfidf_matrix = tfidf_vectorizer.fit_transform(wiki)
tf_model = NearestNeighbors(n_neighbors = 10).fit(tfidf_matrix)

In [48]:
tf_obama = tfidf_matrix.getrow(wiki.index.get_loc('Barack Obama'))
tf_dist = tf_model.kneighbors(tf_obama)[0]
tf_index = tf_model.kneighbors(tf_obama)[1]
new_wiki = wiki.reset_index()['name']
flat_tf_index = tf_index.flatten()
tf_distances = tf_dist.flatten()
series_tf_index = pd.Series(flat_tf_index)
tf_names = series_tf_index.map(new_wiki)
tf_result = pd.DataFrame({'distance':tf_distances, 'name':tf_names})
tf_result

,distance,name
0,0.000000,Barack Obama
1,1.141823,Joe Biden
2,1.196133,Samantha Power
3,1.202502,Hillary Rodham Clinton
4,1.208226,Eric Stern (politician)
5,1.223706,Robert Gibbs
6,1.228785,Eric Holder
7,1.229485,John McCain
8,1.231122,Henry Waxman
9,1.231126,Jeff Sessions


### Let's determine whether this list makes sense.
* Joe Biden is the Vice President when Obama was in power.
* Samantha Power is a member of the Democratic Party who served as the 28th United States Ambassador to the United Nations from 2013 to 2017. 
* Hillary Rodham Clinton is the Secretary of State when Obama was in power.
* John McCain previously served two terms in the United States House of Representatives and was the Republican nominee for president of the United States in the 2008 election, which he lost to Barack Obama.
* Henry Arnold Waxman is an American Democratic politician who served as the U.S. Representative for California's 33rd congressional district from 1975 until 2015.
* Jeff Sessions served as United States Senator from Alabama from 1997 to 2017 during Obama's rule, then he resigned from the position in order to serve in the Trump administration.

Clearly, the results are more plausible with the use of TF-IDF. 😄

## Obama and Hillary Rodham Clinton TF-IDF comparison

Let's take a look at the word vector for Obama and Hillary Rodham Clinton's pages. Notice that TF-IDF representation assigns a weight to each word. This weight captures relative importance of that word in the document. Let us sort the words in Obama's article by their TF-IDF weights; we do the same for Hillary Rodham Clinton's article as well.

In [49]:
def top_words_tf_idf(name):
    # Get the most frequent words in the given person's wikipedia page.
    word = tfidf_matrix.getrow(wiki.index.get_loc(name)).toarray().flatten()
    indices = tfidf_vectorizer.get_feature_names()
    words = pd.Series(word, index = indices)
    words = words.sort_values(ascending=False).reset_index()
    words.rename(columns={'index':'word',0:'count'}, inplace=True)
    return words

In [50]:
obama_tf_idf = top_words_tf_idf('Barack Obama')
obama_tf_idf[0:10]

,word,count
0,obama,0.398184
1,act,0.271722
2,iraq,0.165602
3,law,0.157834
4,control,0.143838
5,us,0.133995
6,ordered,0.133499
7,military,0.130355
8,democratic,0.124986
9,response,0.120199


In [51]:
hillary_tf_idf = top_words_tf_idf('Hillary Rodham Clinton')
hillary_tf_idf[0:10]

,word,count
0,clinton,0.458690
1,she,0.267178
2,lady,0.254605
3,rodham,0.179110
4,us,0.141589
5,secretary,0.137810
6,state,0.130819
7,to,0.130005
8,first,0.128670
9,arkansas,0.127195


Using the **join** operation we learned earlier, try your hands at computing the common words shared by Obama's and Hillary's articles. Sort the common words by their TF-IDF weights in Obama's document.

In [57]:
### Students should write code to answer the question.
combined = obama_tf_idf.join(hillary_tf_idf.set_index('word'),rsuffix='_1', on = 'word')
combined.rename(columns={'count':'Obama','count_1':'Hillary'}, inplace=True)
print(combined[0:5])

      word     Obama   Hillary
0    obama  0.398184  0.080143
1      act  0.271722  0.061526
2     iraq  0.165602  0.074995
3      law  0.157834  0.071477
4  control  0.143838  0.000000


The first 5 words should say: obama, act, iraq, law, control.

### Q3) What is the TF-IDF weight assigned to the word "iraq" for Hillary's page (rounded to the nearest 0.01)? 0.07

## Top 5 word comparison to wikipedia corpus revisited

Among the words that appear in both Barack Obama and Hillary Clinton, take the 5 that have largest weights in Obama. How many of the articles in the Wikipedia dataset contain all of these 5 words?

In [58]:
wiki = wiki.reset_index()
wiki['word_count'] = wiki['text'].apply(word_counter)

### **Q4) Among the words that appear in both Barack Obama and Hillary Clinton pages, take the 5 that have largest weights in Obama. What percentage of the articles in the Wikipedia dataset contain all of these top 5 words? Give answer as a decimal rounded to the nearest 0.0001 - for instance if the percentage found was 0.0817% then state answer as 0.000817.** 0.0033


In [60]:
### Students should write code to answer the question.
common = set(combined.sort_values('Obama', ascending = False)['word'][0:5])
print(common)
def has_top_words(word_count_vector):
    unique_words = word_count_vector.keys()
    if common.issubset(unique_words):
        return True
    else:
        return False
wiki['has_top_words'] = wiki['word_count'].apply(has_top_words)

wiki['has_top_words'].sum()/len(wiki['has_top_words'])

{'iraq', 'act', 'law', 'obama', 'control'}


3.3857561239863894e-05

### **Q5) What can you conclude from observing the differences between the top words existance using the count of words and TF-IDF?**
The 5 most frequently used words for Obama resulted from the count of words are [and, in, of, the, to], and those resulted from TF-IDF are [obama, act, iraq, law, control]. TF-IDF is more reasonable method and penalizes words that are too common.

### **Q6) In a paragrah or two, summerize how the choice of Vectorization (Word Count and TF-IDF) affects the nearest neighbor calculations. Discuss the shortcomings of the bag of words approach (word count) in the context of nearest neighbor document retrieval.**

### **Q7) What is something you would change about the data or the model to improveTF-IDF document retrieval model in general, so that it can find the most similar wikipedia page for any given wikipedia page query? Justify why you think this change may improve the quality of the model.**

**There are many possible correct answers.**

### Well, finally you reached the end of this asignment. Whoo! 🧐
### Wait... in case you don't know, there is another notebook you need to do. Whaaat?
* It's like Erwin Schrödinger's CSE 416 Homework 5.
    * When I am writing this assignment 4 AM in the morning, you are at a state that you both completed and did not complete all the assignment.
    * Now, you can observe yourself and make sure you are in a state that you completed all of Assignment 5. 
    * If not, there is a boosting part of Assignment 5 which you can learn all about boosting 🚀!
    
### Don't forget to make sure you submitted your quiz AND your notebook!  
### You might also want to post to this [really cool online discussion board](https://canvas.uw.edu/courses/1271722/discussion_topics) .